In [1]:
import kagglehub
import numpy as np

# Download latest version
path = kagglehub.dataset_download("rmisra/news-headlines-dataset-for-sarcasm-detection")

print("Path to dataset files:", path)

Path to dataset files: /home/zanchet/.cache/kagglehub/datasets/rmisra/news-headlines-dataset-for-sarcasm-detection/versions/2


In [2]:
import json

sentences = []
labels = []
urls = []
with open(path + "/Sarcasm_Headlines_Dataset.json", 'r') as f:
    for line in f:
        data = json.loads(line)
        sentences.append(data['headline'])
        labels.append(data['is_sarcastic'])
        urls.append(data['article_link'])

In [3]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

2024-12-05 11:00:22.301164: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-05 11:00:22.325445: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733407222.358710   23850 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733407222.371082   23850 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-05 11:00:22.412710: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [4]:
tokenizer = Tokenizer(oov_token='<OOV>')
tokenizer.fit_on_texts(sentences)

In [5]:
word_index = tokenizer.word_index
vocab_size = len(word_index)

In [6]:
sequences = tokenizer.texts_to_sequences(sentences)

In [22]:
pad_lenght = 40
padded = pad_sequences(sequences, padding='post', maxlen=pad_lenght)

In [21]:
len(padded[0])

40

In [9]:
padded.shape

(26709, 40)

In [10]:
validation_percentage = 15

ts = int(padded.shape[0] * (100 - validation_percentage) / 100)

X_train = padded[0:ts]
X_test = padded[ts:]

y_train = labels[0:ts]
y_test = labels[ts:]

In [11]:
X_train.shape

(22702, 40)

In [12]:
X_test.shape

(4007, 40)

In [13]:
import tensorflow as tf


In [14]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, 600, input_length = X_train.shape[1]),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

/home/zanchet/src/Solutions_LAMIA/.venv/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
I0000 00:00:1733407236.903411   23850 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 2278 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3050 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6


In [15]:
num_epochs = 30

In [18]:
history = model.fit(np.array(X_train), np.array(y_train), epochs = num_epochs, validation_data=(np.array(X_test), np.array(y_test)), verbose=2)

Epoch 1/30
710/710 - 7s - 10ms/step - accuracy: 0.9255 - loss: 0.1919 - val_accuracy: 0.8637 - val_loss: 0.3383
Epoch 2/30
710/710 - 4s - 6ms/step - accuracy: 0.9506 - loss: 0.1321 - val_accuracy: 0.8425 - val_loss: 0.4033
Epoch 3/30
710/710 - 4s - 6ms/step - accuracy: 0.9619 - loss: 0.1038 - val_accuracy: 0.8570 - val_loss: 0.4037
Epoch 4/30
710/710 - 4s - 6ms/step - accuracy: 0.9748 - loss: 0.0709 - val_accuracy: 0.8485 - val_loss: 0.4772
Epoch 5/30
710/710 - 4s - 6ms/step - accuracy: 0.9813 - loss: 0.0528 - val_accuracy: 0.8425 - val_loss: 0.5518
Epoch 6/30
710/710 - 4s - 6ms/step - accuracy: 0.9848 - loss: 0.0422 - val_accuracy: 0.8216 - val_loss: 0.6835
Epoch 7/30
710/710 - 4s - 6ms/step - accuracy: 0.9882 - loss: 0.0347 - val_accuracy: 0.8143 - val_loss: 0.7511
Epoch 8/30
710/710 - 4s - 6ms/step - accuracy: 0.9880 - loss: 0.0346 - val_accuracy: 0.8423 - val_loss: 0.6680
Epoch 9/30
710/710 - 4s - 6ms/step - accuracy: 0.9866 - loss: 0.0373 - val_accuracy: 0.8365 - val_loss: 0.7161


In [25]:
sentences = [
    "Hi, I am Lucas, a Computer Engineering student",
    "Niggas are really good at basketball, aren't they?",
    "granny starting to feel spiders in the garden might be real",
    "the weather today is bright and sunny"
]

In [26]:
sequences = tokenizer.texts_to_sequences(sentences)

In [27]:
padded = pad_sequences(sequences, maxlen=pad_lenght, padding="post")

In [28]:
model.predict(padded)

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step


array([[2.2120286e-04],
       [4.5468193e-01],
       [6.3157463e-01],
       [2.6847487e-05]], dtype=float32)